## Data Preprocessing

- Concatenate files into one dataframe for each
- Calculate FPTS
- Resolve name inconsistency

In [2]:
import os
import glob
import time
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from datetime        import datetime
from urllib.request  import urlopen
from bs4             import BeautifulSoup

pd.set_option("display.max_columns",40)

In [9]:
cwd = os.getcwd().replace('/notebooks','')
data_dir = os.path.join(cwd, 'data')
season = '2018-19'

### Game Data from Basketball Reference

In [10]:
#Concatenate all csv files under a directory
def csv_concatenate(folder_path):
    files = glob.glob(folder_path + "/*.csv")
    df_list = []
    for file in tqdm(files):
        df_list.append(pd.read_csv(file, parse_dates=True, infer_datetime_format=True))
    #Fill nan with 0s as some values are empty for percentage points
    df = pd.concat(df_list).fillna(0).reset_index(drop=True)
    return df

In [11]:
def calculate_FPTS(df):
    #Scoring rules based on https://www.draftkings.co.uk/help/rules/4
    multipliers = {'PTS':1, '3P': 0.5, 'TRB':1.25, 'AST':1.5, 'STL':2, 'BLK':2, 'TOV':-0.5}

    fpts_list = []
    
    for i in tqdm(range(df.shape[0])):
        fpts = 0
        doubles_count = 0
        for stat, multiplier in multipliers.items():
            if stat in ['PTS', 'TRB', 'AST', 'STL', 'BLK']:
                if df.loc[i, stat] >= 10:
                    doubles_count += 1
            fpts += df.loc[i, stat]*multiplier
        
        if doubles_count >= 2:
            fpts += 1.5
            
        if doubles_count >= 3:
            fpts += 3
            
        fpts_list.append(fpts) 
        
    return fpts_list

In [12]:
def add_doubles(df):
    
    dd = [0 for i in range(df.shape[0])]
    td = [0 for i in range(df.shape[0])]
    
    for i in tqdm(range(df.shape[0])):
        doubles_count = 0
        check_doubles = ['PTS','TRB', 'AST', 'STL', 'BLK']
        
        for stat in check_doubles:
            if df.loc[i, stat] >= 10:
                doubles_count += 1
        
        if doubles_count >= 2:
            dd[i] = 1
        if doubles_count >= 3:
            td[i] = 1
   
    df['DD'] = dd
    df['TD'] = td

In [13]:
df_games = csv_concatenate(os.path.join(data_dir, 'Games', season))
df_games['FPTS'] = calculate_FPTS(df_games)
add_doubles(df_games)
columns = ['Name', 'Date', 'Team',  'FPTS', 'Home','W', 'W_PTS', 'L', 'L_PTS', 'MP',
           'FG', 'FGA', 'FG_perc', '3P', '3PA', '3P_perc', 'FT', 'FTA', 'FT_perc',
           'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'DD', 'TD', 
           'USG_perc','DRtg','ORtg','AST_perc','DRB_perc','ORB_perc','BLK_perc','TOV_perc','STL_perc','eFG_perc']
df_games = df_games.loc[:, columns]

### Name Standardization 

In [15]:
def parse_name(term):
    search_url = 'https://www.basketball-reference.com/search/search.fcgi?hint=&search={term}&pid=&idx='
    name_url = search_url.format(term=term.replace(' ','+'))
    soup = BeautifulSoup(urlopen(name_url),'html5lib')
    
    #Check if there is ambiguity in the name
    if soup.find('h1').get_text() != 'Search Results':
        return soup.find('h1').get_text()
    
    
    elif (soup.find('div', id='players', class_='current') == None):
        if (len(term.split(' ')) > 2) or ('.' in term):
            #Parse again without periods and with first two names
            new_term = ' '.join(term.replace('.','').split(' ')[:2])
            return parse_name(new_term)
        else:
            return np.nan
                    
    else:        
        items = soup.find('div', id='players', class_='current').find_all('div', class_='search-item-name')
        current_years = (int(season[:4]), int(season[:2]+season[-2:]))
        candidates = []
        
        for item in items:
            name = item.find('a').get_text()

            if '(' not in name:
                candidates.append(name)

            else:
                career = name[name.find('(')+1:name.find(')')].split('-')
                if len(career) == 1:
                    if int(career[0]) in current_years:
                        candidates.append(name[:name.find(' (')])
                else:
                    start = int(career[0])
                    end = int(career[1])

                    for year in current_years: 
                        if year in range(start, end+1):
                            candidates.append(name[:name.find(' (')])
                            break
                            
        if len(candidates) != 0:
            for candidate in candidates:
                if term in candidate:
                    return candidate
            return candidates[0]
        
        else:
            return np.nan
      

In [16]:
def generate_standard_names(df):
    names = list(set(df['Name']))
    standard_names = []
    
    for i, name in enumerate(names):
        standard_name = parse_name(name)
        print(i, standard_name)
        standard_names.append(standard_name)
        time.sleep(1)
    
    return standard_names

In [17]:
def standardize_names(df, standard_names):
    names = list(set(df['Name']))
    
    df = df.dropna().reset_index(drop=True)
    
    diff = [name for name in names if name not in standard_names]
    print('{} items are standardized ...'.format(len(diff)))
    
    names_conversion = {}
    
    for name in tqdm(names):
        if name in diff:
            names_conversion[name] = parse_name(name)
            time.sleep(1)
    
    for i in range(df.shape[0]):
        name = df.loc[i,'Name']
        if name in names_conversion.keys():
            df.loc[i,'Name'] = names_conversion[name]
            

In [18]:
def generate_name_pos(df):
    
    name_pos = {}
    
    for name in set(df['Name']):
        pos = df.loc[(df['Name']==name) & (df['Pos']!=0), 'Pos'].mode()
        if len(pos) != 0:
            name_pos[name] = pos[0]
    
    return name_pos

In [19]:
def fill_pos(df):
    
    name_pos = generate_name_pos(df)
    
    for i in tqdm(range(df.shape[0])):
        if df.loc[i, 'Pos'] == 0:
            name = df.loc[i, 'Name']
            if name in name_pos.keys():
                #print(df.loc[i,'Date'], name)
                df.loc[i, 'Pos'] = name_pos[name]

In [20]:
df_salary = csv_concatenate(os.path.join(data_dir, 'DKSalary', season))

In [21]:
#Takes about 30 mins

standard_names = generate_standard_names(df_salary)
with open(os.path.join(data_dir, 'standard_names','{}.npy'.format(season)), "wb") as fp:
    pickle.dump(standard_names, fp) 

0 Kevon Looney
1 Victor Oladipo
2 Marcin Gortat
3 Jabari Bird
4 Patty Mills
5 Vince Edwards
6 Kent Bazemore
7 Stanley Johnson
8 Jalen Jones
9 Deandre Ayton
10 Mario Hezonja
11 Isaiah Hicks
12 Ben Moore
13 Dwight Howard
14 T.J. McConnell
15 Taj Gibson
16 Luke Kennard
17 Alex Abrines
18 Ivica Zubac
19 Omri Casspi
20 Matthew Dellavedova
21 Gordon Hayward
22 Reggie Bullock
23 Jonas Jerebko
24 Brandon Sampson
25 Thomas Bryant
26 Jusuf Nurkic
27 Markelle Fultz
28 Jerome Robinson
29 Robin Lopez
30 Dragan Bender
31 Tyler Cavanaugh
32 Mike Scott
33 Angel Delgado
34 Damyean Dotson
35 Frank Jackson
36 R.J. Hunter
37 Troy Williams
38 Ben Simmons
39 Terry Rozier
40 Jordan McRae
41 Dante Exum
42 Serge Ibaka
43 Denzel Valentine
44 Isaiah Hartenstein
45 Donte DiVincenzo
46 Andre Iguodala
47 Devin Harris
48 Nicolas Batum
49 Sviatoslav Mykhailiuk
50 Bogdan Bogdanovic
51 Damian Jones
52 Udonis Haslem
53 DeMarre Carroll
54 Ryan Arcidiacono
55 Gary Payton II
56 Darren Collison
57 Timofey Mozgov
58 Nick You

459 Isaiah Canaan
460 Yante Maten
461 Justin Patton
462 Kostas Antetokounmpo
463 Josh Jackson
464 Collin Sexton
465 Alex Caruso
466 Myles Turner
467 Dirk Nowitzki
468 Tyler Dorsey
469 PJ Dozier
470 Kentavious Caldwell-Pope
471 Dorian Finney-Smith
472 Chandler Hutchison
473 Malik Beasley
474 Meyers Leonard
475 Jerryd Bayless
476 Tim Frazier
477 Royce O'Neale
478 Ed Davis
479 Glenn Robinson III
480 Sindarius Thornwell
481 Edmond Sumner
482 Stephan Hicks
483 George Hill
484 Maxi Kleber
485 Kawhi Leonard
486 Dzanan Musa
487 Gerald Green
488 Yogi Ferrell
489 Bruce Brown
490 Isaiah Briscoe
491 De'Aaron Fox
492 Jayson Tatum
493 Chasson Randle
494 Lance Thomas
495 Devin Robinson
496 Taurean Prince
497 Jimmy Butler
498 C.J. Miles
499 P.J. Tucker
500 DeAndre' Bembry
501 Wade Baldwin
502 Clint Capela
503 Tyler Lydon
504 Andre Drummond
505 Chris Paul
506 Marcus Derrickson
507 Khris Middleton
508 Alize Johnson
509 Kevin Durant
510 Richaun Holmes
511 Rodions Kurucs
512 Lonnie Walker
513 Allonzo Trie

In [22]:
with open(os.path.join(data_dir, 'standard_names', '{}.npy'.format(season)), "rb") as fp:
    standard_names = pickle.load(fp)

In [23]:
standardize_names(df_salary, standard_names)

24 items are standardized ...


In [24]:
standardize_names(df_games, standard_names)

17 items are standardized ...


In [25]:
fill_pos(df_salary)

In [26]:
df = pd.merge(df_salary.drop('Team', axis=1), df_games, on=['Name', 'Date'], how='inner')
df = df[df['Pos']!=0].sort_values(by=['Date','Team']).reset_index(drop=True)

In [27]:
# Add Value variable
df['Value'] = df['FPTS']/(df['Salary']/1000)
df['Value'] = df['Value'].replace(np.inf, 0).replace(-np.inf, 0)

In [28]:
columns = ['Name', 'Pos', 'Date', 'Team', 'FPTS', 'Value', 'Home', 'W', 'W_PTS', 'L', 'L_PTS', 'MP',
           'FG', 'FGA', 'FG_perc', '3P', '3PA', '3P_perc', 'FT', 'FTA', 'FT_perc', 
           'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'DD', 'TD', 
           'USG_perc','DRtg','ORtg','AST_perc','DRB_perc','ORB_perc','BLK_perc','TOV_perc','STL_perc','eFG_perc']

columns = columns[:3] + ['Salary', 'Starter'] + columns[3:]

df = df.loc[:, columns]

df.to_csv(os.path.join(data_dir, 'Dataframes', 'clean','df_{}.csv'.format(season)), index=False)